In [11]:
import torch

att = torch.ones((1, 8, 8), dtype=torch.bool) * 3

matrix = torch.ones((8, 8), dtype=torch.bool)
matrix.fill_diagonal_(0)
matrix[1:].fill_diagonal_(0)

att[:, matrix] = 0

att

tensor([[[3, 0, 0, 0, 0, 0, 0, 0],
         [3, 3, 0, 0, 0, 0, 0, 0],
         [0, 3, 3, 0, 0, 0, 0, 0],
         [0, 0, 3, 3, 0, 0, 0, 0],
         [0, 0, 0, 3, 3, 0, 0, 0],
         [0, 0, 0, 0, 3, 3, 0, 0],
         [0, 0, 0, 0, 0, 3, 3, 0],
         [0, 0, 0, 0, 0, 0, 3, 3]]])

In [30]:
import torch


class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return torch.tensor(self.dataset[idx]["source_tokens"]), torch.tensor(self.dataset[idx]["target_tokens"])


class NGram(torch.nn.Module):
    def __init__(self, n, vocab_size):
        super(NGram, self).__init__()
        self.n = n
        self.embeddings = torch.nn.Embedding(vocab_size, 128)
        self.linear = torch.nn.Linear(128 * n, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [33]:
loader = torch.utils.data.DataLoader(TranslationDataset(dataset["train"]), batch_size=32)

def train():
    n = 3
    model = NGram(n, 10000)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):
        total_loss = 0
        for x, y in loader:
            print(x.shape)
            print(y.shape)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            print(loss.item())
        print(total_loss)
        
train()

torch.Size([32, 64])
torch.Size([32, 64])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8192 and 384x10000)